# 🎛️ VGAIN SCAN ANALYSIS

## Three Analysis Options

| **Mode** | **Function** | **Use Case** |
|----------|--------------|--------------|
| **🔍 Step-by-Step** | Manual cell-by-cell | **Interactive debugging**<br/>Run analysis step-by-step in this notebook |
| **⚡ Single Vgain** | `single_vgain_analysis()` | **Single run automation**<br/>Automatic analysis for one run |
| **🚀 Full Scan** | `channel_vgain_scan_analysis()` | **Batch processing**<br/>Automatic multiple run analysis for same channel and bias |



INPUT PARAMETERS

Common to all analysis

In [ ]:
%load_ext autoreload
%autoreload 2

from waffles.coldboxVD.november_25.ab_coldbox.ab_utils import *

In [ ]:
# RUN INPUT PARAMETERS
membrane = 'M3'
channel = 29
bias = '31V47' #4.5 OV
vgain = 800

In [ ]:
# ANALYSIS INPUT PARAMETERS

# dict_params = vgain_analysis_parameters(vgain) # automatically computed (ok for M3 -29)

## or manually set

dict_params_all = {800: { 'baseline_timeticks_limit': 380,
                            'deviation_from_baseline': 0.6,
                            'heatmap_min': -170,
                            'heatmap_max': 550,
                            'adcs_threshold': 60,
                            'n_std_baseline': 1,
                            "max_peaks": 7,                
                            "prominence": 0.5,            
                            "initial_percentage": 0.1,     
                            "percentage_step": 0.02,       
                            "ch_span_fraction_around_peaks": 0.05},}

dict_params = dict_params_all[vgain]


In [ ]:
# DEFAULT INPUT PARAMETERS
coldbox_folder = "/eos/experiment/neutplatform/protodune/experiments/ColdBoxVD/November2025run/spy_buffer/VGAIN_SCAN"
input_file = f"{coldbox_folder}/{membrane}/vgain_scan_{membrane}_DVbias_{bias}/vgain_{vgain}/channel_{channel}.dat"

## 🔍 **Step-by-Step Analysis**


In [ ]:
wfset_original = create_waveform_set_from_spybuffer(filename=input_file, WFs=40000, length=1024, config_channel=channel)
# plotting_overlap_wf(wfset_original, index_list=[1,2])
# plotting_overlap_wf(wfset_original, n_wf=5)

REMOVING BASELINE

In [ ]:
# BASELINE ANALYSIS + removing 

baseliner_input_parameters = IPDict({
            'baseline_limits': (0,dict_params['baseline_timeticks_limit']),
            'std_cut': 1.,
            'type': 'mean'
        })

checks_kwargs = IPDict({
    'points_no': wfset_original.points_per_wf
})

baseline_analysis_label = 'baseline'

_ = wfset_original.analyse(
    baseline_analysis_label,
    WindowBaseliner,
    baseliner_input_parameters,
    checks_kwargs=checks_kwargs,
    overwrite=True
)

In [ ]:
wfset_original.apply(subtract_baseline, baseline_analysis_label, show_progress=False)
# plotting_overlap_wf(wfset_original, n_wf=2)

In [ ]:
# Dummy analysis for later - it sets the baseline to 0 always

null_baseline_analysis_label = 'null_baseliner'
_ = wfset_original.analyse(
            null_baseline_analysis_label,
            StoreWfAna,
            {'baseline': 0.},
            overwrite=True
        )

STARTING THE FILTERING PROCEDURE

By using the persistance plot, try to find a good sub wfset for the finger plot

In [ ]:
# No filter 

persistance_plot_helper(wfset_original, channel, ymin = dict_params['heatmap_min'], ymax = dict_params['heatmap_max'], adc_bins = 1000)

In [ ]:
# First step - remove waveforms which goes below and up some adcs thresholds in the baseline region (similar to coarse_selection_for_led_calibration from Julio's code)

wfset_1 = WaveformSet.from_filtered_WaveformSet(wfset_original, adcs_threshold_filter, time_range = [0,dict_params['baseline_timeticks_limit']], adcs_minimum_threshold=-dict_params['adcs_threshold'], adcs_maximum_threshold=dict_params['adcs_threshold'])
persistance_plot_helper(wfset_1, channel, ymin = dict_params['heatmap_min'], ymax = dict_params['heatmap_max'], adc_bins = 1000)

In [ ]:
# Second step - look at baseline std distribution to remove noisy waveforms -

average_baseline_std = compute_average_baseline_std(wfset_1, baseline_analysis_label)
wfset_2 = WaveformSet.from_filtered_WaveformSet(wfset_1, baseline_std_selection, baseline_analysis_label, average_baseline_std, dict_params['n_std_baseline'])

persistance_plot_helper(wfset_2, channel, ymin = dict_params['heatmap_min'], ymax = dict_params['heatmap_max'], adc_bins = 1000)

In [ ]:
print(f"Original wfset: {len(wfset_original.waveforms)}")
print(f"Adcs cut 1: {len(wfset_1.waveforms)}")
print(f"Std cut: {len(wfset_2.waveforms)}")

wfset_filtered = wfset_2

INTEGRATION WINDOW DEFINITION

Computing the mean waveform 

In [ ]:
mean_wf = wfset_filtered.compute_mean_waveform()

plt.figure()
plt.plot(np.array(range(0,1024)), mean_wf.adcs, label="Mean wf")
plt.xlabel("Time ticks")
plt.ylabel("Adcs")
plt.title(f"Mean waveform")
plt.show()

In [ ]:
aux_limits = get_pulse_window_limits(
                    adcs_array = -mean_wf.adcs,
                    baseline = 0,
                    deviation_from_baseline = dict_params['deviation_from_baseline'],
                    get_zero_crossing_upper_limit = False
                )

print(aux_limits)

In [ ]:
federico_limits = (382, 406)

plt.figure()
plt.plot(np.array(range(0,1024)), mean_wf.adcs, label="Mean wf")
plt.axvline(
    aux_limits[0],
    linestyle="--",
    color="red",
    linewidth=1,
    label=f"My LL = {aux_limits[0]} ({dict_params['deviation_from_baseline']:.1f} σ)"
)

plt.axvline(
    aux_limits[1],
    linestyle="--",
    color="blue",
    linewidth=1,
    label=f"My UL = {aux_limits[1]} ({dict_params['deviation_from_baseline']:.1f} σ)"
)

plt.axvline(
    federico_limits[0],
    linestyle="-",
    color="red",
    linewidth=1,
    label=f"Federico LL = {federico_limits[0]}"
)

plt.axvline(
    federico_limits[1],
    linestyle="-",
    color="blue",
    linewidth=1,
    label=f"Federico UL = {federico_limits[1]}"
)

plt.legend()
plt.xlabel("Time ticks (AU)")
plt.ylabel("Adcs")
plt.title(f"Mean waveform")
# plt.xlim(350,500)
plt.show()

In [ ]:
# Decide if you want to use your or Federico's limits for integration
aux_limits = federico_limits

INTEGRATION ANALYSIS

In [ ]:
integration_analysis_label = 'integration_analysis'

integrator_input_parameters = IPDict({
        'baseline_analysis': null_baseline_analysis_label,
        'inversion': False,
        'int_ll': aux_limits[0],
        'int_ul': aux_limits[1],
        'amp_ll': aux_limits[0],
        'amp_ul': aux_limits[1]
    })

checks_kwargs = IPDict({'points_no': wfset_filtered.points_per_wf})

_ = wfset_filtered.analyse(
    integration_analysis_label,
    WindowIntegrator,
    integrator_input_parameters,
    checks_kwargs=checks_kwargs,
    overwrite=True
)

CALIBRATION HISTOGRAM 

In [ ]:
hist_domain, hist_nbins, hist_bins_width = auto_histogram(wfset_filtered, integration_analysis_label, show_results=True)

In [ ]:
my_grid = coldbox_single_channel_grid(wfset_filtered, config_channel=channel)

my_grid.compute_calib_histos(
            bins_number=hist_nbins, 
            domain=hist_domain, 
            variable='integral',
            analysis_label=integration_analysis_label
        )

fit_peaks_of_ChannelWsGrid(
        my_grid,
        max_peaks=dict_params['max_peaks'], 
        prominence=float(dict_params['prominence']), 
        initial_percentage=dict_params['initial_percentage'],
        percentage_step=dict_params['percentage_step'],
        return_last_addition_if_fail=True,
        fit_type='multigauss_iminuit',
        weigh_fit_by_poisson_sigmas=True,
        ch_span_fraction_around_peaks=dict_params['ch_span_fraction_around_peaks']
    )

# # If you want to play on parameters manually
# fit_peaks_of_ChannelWsGrid(
#     my_grid,
#     max_peaks=7,
#     prominence=0.5,
#     initial_percentage=0.1,
#     percentage_step=0.02,
#     return_last_addition_if_fail=True,
#     fit_type='multigauss_iminuit',
#     weigh_fit_by_poisson_sigmas=True,
#     ch_span_fraction_around_peaks=0.05
# )


fig = plot_ChannelWsGrid(
    my_grid, 
    mode='calibration',
    plot_peaks_fits=True,           
    plot_sum_of_gaussians=True      
)

fig.show()

output_parameters = print_correlated_gaussians_fit_parameters(my_grid, federico_conversion=True, show=True)


-------------


## ⚡ **Single Vgain Analysis**


In [ ]:
single_vgain_analysis(membrane = membrane, 
                    channel = channel, 
                    bias = bias, 
                    vgain = vgain,
                    dict_params = dict_params, # you have to give it!!
                    save_pdf = False,
                    show = True)

-------------


## 🚀 **Full Vgain Scan**


In [ ]:
channel_vgain_scan_analysis(
                    membrane = membrane, 
                    channel = channel, 
                    bias = bias,
                    vgain_list = [800,900], #range(500, 2301, 100) or dict_params_all.keys()
                    #external_dict_paramas = dict_params_all, # OPTIONAL!!! if it is given, it uses params form the given out. else outomatically computed by vgain_analysis_parameters()
                    federico_limits = True,
                    federico_conversion = True,
                    show = False,
                    save_pdf = True,
                    )

Other stuff...

In [ ]:
# Correct values for 'heatmap_max', 'heatmap_min', 'adcs_threshold', 'n_std_baseline' for M3 29 channel and 31V47 bias

dict_input_M3_29_31V47= { 
                    500 : {'heatmap_max': 700, 'heatmap_min':-200, 'adcs_threshold': 90, 'n_std_baseline':1,},
                    600 : {'heatmap_max': 650, 'heatmap_min':-190, 'adcs_threshold': 80, 'n_std_baseline':1,},
                    700 : {'heatmap_max': 600, 'heatmap_min':-180, 'adcs_threshold': 70, 'n_std_baseline':1,},
                    800 : {'heatmap_max': 550, 'heatmap_min':-170, 'adcs_threshold': 60, 'n_std_baseline':1},
                    900 : {'heatmap_max': 500, 'heatmap_min':-160, 'adcs_threshold': 50, 'n_std_baseline':1},
                    1000 : {'heatmap_max': 450, 'heatmap_min':-150, 'adcs_threshold': 45, 'n_std_baseline':1},
                    1100 : {'heatmap_max': 400, 'heatmap_min':-140, 'adcs_threshold': 40, 'n_std_baseline':1},
                    1200 : {'heatmap_max': 350, 'heatmap_min':-130, 'adcs_threshold': 35, 'n_std_baseline':1},
                    1300 : {'heatmap_max': 300, 'heatmap_min':-120, 'adcs_threshold': 30, 'n_std_baseline':1},
                    1400 : {'heatmap_max': 250, 'heatmap_min':-110, 'adcs_threshold': 27, 'n_std_baseline':1},
                    1500 : {'heatmap_max': 200, 'heatmap_min':-100, 'adcs_threshold': 24, 'n_std_baseline':1},
                    1600 : {'heatmap_max': 150, 'heatmap_min':-90, 'adcs_threshold': 21, 'n_std_baseline':1},
                    1700 : {'heatmap_max': 100, 'heatmap_min':-80, 'adcs_threshold': 18, 'n_std_baseline':1},
                    1800 : {'heatmap_max': 100, 'heatmap_min':-70, 'adcs_threshold': 15, 'n_std_baseline':1},
                    1900 : {'heatmap_max': 100, 'heatmap_min':-60, 'adcs_threshold': 12, 'n_std_baseline':1.1},
                    2000 : {'heatmap_max': 100, 'heatmap_min':-50, 'adcs_threshold':11, 'n_std_baseline':1.1 },
                    # 2100 : {'heatmap_max': 100, 'heatmap_min':-200, 'adcs_threshold': 10, 'n_std_baseline':1.15}, # NO FIT
                    # 2200 : {'heatmap_max': 100, 'heatmap_min':-200, 'adcs_threshold': 9, 'n_std_baseline':1.2}, # NO FIT
                    # 2300 : {'heatmap_max': 100, 'heatmap_min':-200, 'adcs_threshold': 8, 'n_std_baseline':1.2}, # NO FIT
                     }